# Calculate and save frozen moist static energy

Note: In SAM's simple single moment microphysics scheme, there is a simple temperature-dependent partitioning between liquid and ice. We need to calculate ice mixing ratios (non-precipitating and precipitating) from these equations (Khairoutdinov and Randall 2003):

$q_c = \omega_n q_n$ *(cloud water)* 

$q_i = (1-\omega_n) q_n$ *(cloud ice)* 

$q_r = \omega_p q_p$ *(precipitating liquid, "rain")*

$q_s = (1-\omega_p)(1-\omega_g)q_p$ *("snow")*

$q_g = (1-\omega_p)\omega_g q_p$ *("graupel")*

With the partition functions $\omega_m$ depending on temperature alone:

$\omega_m = \textrm{max}[0, \textrm{min}(1, \frac{T-T_{00m}}{T_{0m} - T_{00m}})]$

with 

$T_{0n} = 285$ *(this is different from KR03)*
$T_{0p} = 283.16 \\
T_{0g} = 283.16\\ 
T_{00n} = 253.16 \\
T_{00p} = 268.16 \\ 
T_{00g} = 223.16$

In [1]:
import xarray as xr
import glob
import numpy as np
from datetime import datetime
import os

In [2]:
# pre-defined fields

author = # your name 
email = # your email address 

simset = 'rcemip-large'

dirout = 'FMSE'

T_0n = 285
T_0p = 283.16
T_00n = 253.16
T_00p = 268.16

cp = 1006
g = 9.81
Lv = 2.5e6
Lf = 3.34e5

cases = ['nz_32','nz_64','nz_128','nz_256']

if simset == 'rcemip-small':
    outstr1 = '_99km_300K_'
    nxny = '_128_'
else:
    outstr1 = '_1536km_300K_'
    nxny = '_512_'
    
    cases = cases[:2]
    
fpath = '/ocean/projects/atm200007p/ajenney/' + simset + '/'

In [3]:
# today's date as a string

today = datetime.today().strftime('%Y-%m-%d')

In [ ]:
# Loop through cases
for icase, case in enumerate(cases):
    
    print('working on ' + case + '...')

    # Load the background pressure file
    pfile = glob.glob(fpath + case + '/OUT_3D/p/*.nc')[0]
    p_background = xr.open_dataset(pfile).p*100 # convert from hPa to Pa

    # List files to get timestamps
    files_ta = np.sort(glob.glob(fpath + case + '/OUT_3D/TABS/*.nc'))
    files_qn = np.sort(glob.glob(fpath + case + '/OUT_3D/QN/*.nc'))
    files_qp = np.sort(glob.glob(fpath + case + '/OUT_3D/QP/*.nc'))
    files_qv = np.sort(glob.glob(fpath + case + '/OUT_3D/QV/*.nc'))
    files_p = np.sort(glob.glob(fpath + case + '/OUT_3D/PP/*.nc'))

    # loop through time steps, calculate and save frozen MSE

    try:
        os.mkdir(fpath + case + '/OUT_3D/' + dirout)
    except:
        print('directory already exists!')

    for ifile in range(len(files_ta)):

        tstr = files_ta[ifile][-13:-3] # time string

        print(str(ifile) + '/' + str(len(files_ta)))

        filename_out = (fpath + case + '/OUT_3D/' + dirout + '/RCE_' + simset + outstr1 +
                        case + nxny + dirout + '_' + tstr + '.nc')

        if os.path.exists(filename_out): 
                print('file exists! Skipping this one')
                continue

        # Load corresponding TABS, QN, QP, QV, PP file
        ta = xr.open_dataset(files_ta[ifile]).TABS
        qn = xr.open_dataset(files_qn[ifile]).QN
        qp = xr.open_dataset(files_qp[ifile]).QP
        qv = xr.open_dataset(files_qv[ifile]).QV
        p = xr.open_dataset(files_p[ifile]).PP + p_background

        # ---- Calculate cloud ice mixing ratio from equation ----
        omega_n = xr.ufuncs.maximum(0, 
                                    xr.ufuncs.minimum(1, 
                                                      (ta - T_00n)/
                                                      (T_0n-T_00n)))

        qi = (1-omega_n)*qn / 1000 # from g/kg to kg/kg 

        del omega_n
        del qn

        # ---- Calculate precipitating ice mixing ratio from equation ----
        omega_p = xr.ufuncs.maximum(0, 
                                    xr.ufuncs.minimum(1, 
                                                      (ta - T_00p)/
                                                      (T_0p-T_00p)))

        qpi = (1-omega_p)*qp / 1000 # from g/kg to kg/kg 

        del omega_p
        del qp

        # ---- Calculate frozen moist static energy ----
        # cpT + gz + Lv*qv - Lf*qi

        h_i = (cp * ta + 
               g * ta.z + 
               Lv * qv / 1000 - # from g/kg to kg/kg 
               Lf * (qi + qpi))

        h_i.attrs['long_name'] = 'Frozen moist static energy'
        h_i.attrs['units'] = 'J/kg'
        h_i.attrs['description'] = 'cpT + gz + Lv*qv - Lf*qi'

        del qv

        # ---- Calculate the mass-weighted vertical integral
        dp = xr.ufuncs.fabs(p.diff(dim='z'))
        h_i_vint = (1/g) * (h_i[:,:-1,:,:] * dp).sum(dim='z')

        h_i_vint.attrs['long_name'] = 'Vertically integrated frozen moist static energy'
        h_i_vint.attrs['units'] = 'J/m2'
        h_i_vint.attrs['description'] = 'mass weighting calculated as sum(fmse*dp)/g'


        # Save to netcdf file
        ds_out = xr.Dataset(
                            data_vars=dict(fmse=h_i,
                                          fmse_vint=h_i_vint),
                            attrs=dict(history='FMSE calculated on ' + 
                                               today + ' by ' + author + ': ' + 
                                               email)
                                      )
        ds_out.to_netcdf(filename_out)

        del h_i
        del h_i_vint
        del ds_out
        del ta
        del p